In [1]:
%pwd

'/home/zishan/Documents/projects/commodity-price-forecasting/notebooks'

In [2]:
from pathlib import Path
import re

import pandas as pd

In [3]:
years = set()
months = set()
days = set()

path = Path('../data/tcb_files')
for file in path.glob('*.xlsx'):
    year, month, day = map(int, file.name.split('-')[:3])
    years.add(year)
    months.add(month)
    days.add(day)

print("unique years\n", years)
print("unique_months\n", months)
print("unique_days\n", days)

unique years
 {2021, 2022, 2023, 2024, 2025}
unique_months
 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}
unique_days
 {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}


**the file names hold the date of the report in the form yyyy-mm-dd and the values seem to be alright**

In [4]:
date_counts = {}
path = Path('../data/tcb_files')
for file in path.glob('*.xlsx'):
    date = '-'.join(file.name.split('-')[:3])
    if date not in date_counts:
        date_counts[date] = 1
    else:
        date_counts[date] += 1

for date, count in date_counts.items():
    if count > 1:
        print(date, count)

2024-07-28 6
2023-11-07 2
2021-05-24 6
2024-11-07 2
2024-08-11 11
2024-10-20 3
2024-01-21 2
2025-03-24 4
2025-06-29 2
2025-02-18 2
2024-12-15 3
2024-04-08 4
2024-04-16 2
2025-01-19 2
2024-08-20 6
2023-10-25 2
2025-06-25 2
2022-05-17 2
2023-11-29 2
2025-09-09 7
2023-02-12 2
2022-02-23 2
2025-06-02 2
2025-08-17 4
2024-06-10 2
2024-05-16 2
2024-11-17 3
2023-12-30 2
2023-11-15 2
2023-03-02 2
2024-11-02 2
2021-09-05 2
2024-11-10 3
2024-09-23 2
2025-01-05 2
2024-07-14 2
2025-01-12 2
2024-10-16 2
2024-12-30 2
2023-04-04 2
2023-03-13 2
2023-12-20 2
2024-05-27 2
2024-01-23 2
2025-04-05 2
2023-10-16 2


**date given in the file names doesn't seem to be reliable. have to figure out different way of extracting date**

In [32]:
file_lengths = {}
for file in path.glob("*.xlsx"):
    try:
        df = pd.read_excel(file, engine='openpyxl')
        length = len(df)
        if length in file_lengths:
            file_lengths[length] += 1
        else:
            file_lengths[length] = 1
    except Exception as e:
        print(f"couldn't read file {file.name}", e)
print("file lengths:\n", file_lengths)

couldn't read file 2023-10-16-09-31-e3cf69220d9c02c8e32af904e03fbdd9.xlsx File is not a zip file
file lengths:
 {121: 63, 125: 87, 118: 49, 123: 68, 120: 62, 126: 63, 135: 9, 132: 18, 117: 33, 127: 48, 129: 40, 128: 55, 116: 30, 119: 63, 124: 65, 131: 50, 122: 78, 130: 51, 96: 36, 134: 12, 115: 19, 137: 1, 112: 7, 110: 1, 113: 7, 138: 6, 140: 2, 133: 23, 136: 7, 114: 12, 111: 2, 139: 4, 141: 1}


In [9]:
possible_dates = []
for file in path.glob("*.xlsx"):
    try:
        df = pd.read_excel(file, engine='openpyxl')
        twelfth_col = df.iloc[:, 11]
        first_non_null = twelfth_col.dropna().iloc[0] if not twelfth_col.dropna().empty else None
        if first_non_null is not None:
            possible_dates.append(first_non_null)
    except Exception as e:
        print(f"couldn't read file {file.name}", e)

possible_dates = pd.Series(possible_dates)

# Find values that can't be parsed as dates
def find_non_date_values(series):
    non_date_values = []
    for value in series:
        try:
            pd.to_datetime(value, errors='raise')
        except (ValueError, TypeError):
            non_date_values.append(value)
    return non_date_values

non_date_values = find_non_date_values(possible_dates)

print(f"Total values: {len(possible_dates)}")
print(f"Non-date values found: {len(non_date_values)}")
print(f"Non-date values: {set(non_date_values)}")

couldn't read file 2023-10-16-09-31-e3cf69220d9c02c8e32af904e03fbdd9.xlsx File is not a zip file
Total values: 1061
Non-date values found: 36
Non-date values: {'টিসিবির মূল্য'}


In [39]:
pd.set_option('display.max_rows', 150)
df = pd.read_excel(list(path.glob("*.xlsx"))[0])
df

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,,NaN,NaN,NaN,NaN,ট্রেডিং কর্পোরেশন অব বাংলাদেশ,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,www.tcb.gov.bd,NaN,NaN,NaN,NaN,NaN,NaN
2,,NaN,NaN,NaN,NaN,ঢাকা মহানগরীর নিত্য প্রয়োজনীয় দ্রব্যের খুচরা ব...,NaN,NaN,NaN,NaN,সিরিয়াল নংঃ ১৮৯,NaN
3,NaN,NaN,NaN,NaN,NaN,"শুক্রবার ০১ আগষ্ট ২০২৫ খ্রিঃ, ১৭ শ্রাবণ ১৪৩২ ব...",NaN,NaN,NaN,NaN,NaN,NaN
4,স্মারক নং-২৬.০৫.০০০০.০১৭.৩১.০১.২৫-৫৪৪,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,তারিখঃ,2025-08-01 00:00:00
5,পণ্যের নাম,মাপের একক,অদ্যকার মূল্য(টাকায়),NaN,১সপ্তাহ পূর্বের মূল্য(টাকায়),NaN,১ মাস পূর্বের মূল্য(টাকায়),NaN,মাসিক মূল্যের,১বছর পূর্বের মূল্য(টাকায়),NaN,বাৎসরিক মূল্যের
6,NaN,NaN,2025-08-01 00:00:00,NaN,2025-07-25 00:00:00,NaN,2025-07-01 00:00:00,NaN,হ্রাস/বৃদ্ধি(%),2024-08-01 00:00:00,NaN,হ্রাস/বৃদ্ধি(%)
7,চাল,NaN,সর্বনিম্ন,সর্ব্বোচ্চ,সর্বনিম্ন,সর্ব্বোচ্চ,সর্বনিম্ন,সর্ব্বোচ্চ,(+)/(-),সর্বনিম্ন,সর্ব্বোচ্চ,(+)/(-)
8,চাল সরু (নাজির/মিনিকেট),প্রতি কেজি,75,85,75,85,75,85,0,60,78,15.942029
9,চাল (মাঝারী)পাইজাম/আটাশ,প্রতি কেজি,60,75,60,70,60,65,8,54,58,20.535714


**example file inspection yeilds following patterns:**
- useful information starts from a line including 'চাল' in the first column of the row and ends before a line starting with 'যে সকল বাজার হতে তথ্য সংগ্রহ করা হয়েছে'
- there are past values given for reference but useful columns are first 4 columns: item, unit, low_price, high_price
- items are divided by category which is on a row in the first column followed by items in that category. 
- there seems to be 8 category total: চাল, আটা/ময়দা, ভোজ্য তেল, ডাল, মসলা, মাছ ও গোশত, গুড়া দুধ(প্যাকেটজাত), বিবিধ in that order
- actual items with prices will all have first 4 columns filled

In [87]:
def extract_date_from_filename(filename: Path) -> str | None:
    match = re.match(r"(\d{4}-\d{2}-\d{2})", filename.stem)
    if match:
        return match.group(1)
    return None


def process_tcb_file(file_path: Path) -> pd.DataFrame:
    try:
        df_raw = pd.read_excel(file_path, header=None, engine="openpyxl")

        start_idx: int | None = None
        end_idx: int | None = None

        for idx, row in df_raw.iterrows():
            first_col = str(row[0]).strip() if pd.notna(row[0]) else ""

            if start_idx is None and "চাল" in first_col and "সর্বনিম্ন" not in first_col:
                start_idx = idx

            if (
                start_idx is not None
                and "যে সকল বাজার হতে তথ্য সংগ্রহ করা হয়েছে" in first_col
            ):
                end_idx = idx
                break

        if start_idx is None or end_idx is None:
            return pd.DataFrame()

        df_section = df_raw.iloc[start_idx:end_idx].copy()
        df_section = df_section.reset_index(drop=True)

        categories: list[str] = [
            "চাল",
            "আটা/ময়দা",
            "ভোজ্য তেল",
            "ডাল",
            "মসলাঃ",
            "মাছ ও গোশত:",
            "গুড়া দুধ(প্যাকেটজাত)",
            "বিবিধঃ",
        ]

        current_category: str | None = None
        extracted_data: list[dict[str, str | None]] = []

        for idx, row in df_section.iterrows():
            col1 = str(row[0]).strip() if pd.notna(row[0]) else ""
            col2 = str(row[1]).strip() if pd.notna(row[1]) else ""
            col3 = str(row[2]).strip() if pd.notna(row[2]) else ""
            col4 = str(row[3]).strip() if pd.notna(row[3]) else ""

            if col1 in categories and (not col2 or col2 == "nan"):
                current_category = col1
                continue

            if (col1 == "nan" or not col1) and (
                "সর্বনিম্ন" in col3 and "সর্ব্বোচ্চ" in col4
            ):
                continue

            if (
                current_category
                and col1
                and col2
                and col3
                and col4
                and col1 != "nan"
                and col2 != "nan"
                and col3 != "nan"
                and col4 != "nan"
                and not col1.isdigit()
                and col1 not in categories
            ):

                if "=" in str(col3) or "=" in str(col4):
                    continue

                date = extract_date_from_filename(file_path)

                item_data: dict[str, str | None] = {
                    "date": date,
                    "category": current_category,
                    "item": col1.strip(),
                    "unit": col2.strip(),
                    "low_price": col3.strip(),
                    "high_price": col4.strip(),
                }
                extracted_data.append(item_data)

        return pd.DataFrame(extracted_data)

    except Exception as e:
        return pd.DataFrame()


def process_all_files(data_folder: str = 'data/tcb_files') -> pd.DataFrame:
    all_data: list[pd.DataFrame] = []
    path = Path(data_folder)

    files_processed: int = 0
    files_with_data: int = 0
    files_without_data: int = 0
    files_error: int = 0

    for file in path.glob("*.xlsx"):
        try:
            df_processed = process_tcb_file(file)

            if df_processed is not None and not df_processed.empty:
                all_data.append(df_processed)
                files_with_data += 1
            else:
                files_without_data += 1

            files_processed += 1

        except Exception:
            files_error += 1

    print("=== PROCESSING SUMMARY ===")
    print(f"Total files processed: {files_processed}")
    print(f"Files with data extracted: {files_with_data}")
    print(f"Files with NO data extracted: {files_without_data}")
    print(f"Files with errors: {files_error}")
    
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        return final_df
    else:
        return pd.DataFrame(
            columns=["date", "category", "item", "unit", "low_price", "high_price"]
        )


In [88]:
data_folder = "../data/tcb_files"
final_data = process_all_files(data_folder)

if final_data is not None:
    print("\nFirst few rows of extracted data:")
    print(final_data.head(20))
    print(f"\nData shape: {final_data.shape}")
    

=== PROCESSING SUMMARY ===
Total files processed: 1073
Files with data extracted: 1036
Files with NO data extracted: 37
Files with errors: 0

First few rows of extracted data:
          date   category                          item              unit  \
0   2025-08-01        চাল       চাল সরু (নাজির/মিনিকেট)        প্রতি কেজি   
1   2025-08-01        চাল       চাল (মাঝারী)পাইজাম/আটাশ        প্রতি কেজি   
2   2025-08-01        চাল  চাল (মোটা)/স্বর্ণা/চায়না ইরি        প্রতি কেজি   
3   2025-08-01   আটা/ময়দা               আটা সাদা (খোলা)        প্রতি কেজি   
4   2025-08-01   আটা/ময়দা                 আটা (প্যাকেট)  প্রতি কেজি প্যাঃ   
5   2025-08-01   আটা/ময়দা                   ময়দা (খোলা)        প্রতি কেজি   
6   2025-08-01   আটা/ময়দা                ময়দা (প্যাকেট)  প্রতি কেজি প্যাঃ   
7   2025-08-01  ভোজ্য তেল              সয়াবিন তেল (লুজ)       প্রতি লিটার   
8   2025-08-01  ভোজ্য তেল             সয়াবিন তেল (বোতল)           ৫ লিটার   
9   2025-08-01  ভোজ্য তেল             সয়াবিন তেল (বোতল

**36 files seem to have no coherent structures to the data so no data was extracted**

In [82]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55043 entries, 0 to 55042
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        55043 non-null  object
 1   category    55043 non-null  object
 2   item        55043 non-null  object
 3   unit        55043 non-null  object
 4   low_price   55043 non-null  object
 5   high_price  55043 non-null  object
dtypes: object(6)
memory usage: 2.5+ MB


In [85]:
for arr in final_data.groupby('category')['item'].unique():
    print(arr)

['আটা সাদা (খোলা)' 'আটা (প্যাকেট)' 'ময়দা (খোলা)' 'ময়দা (প্যাকেট)']
['ডানো' 'ডিপ্লোমা (নিউজিল্যান্ড)' 'ফ্রেশ' 'মার্কস']
['চাল সরু (নাজির/মিনিকেট)' 'চাল (মাঝারী)পাইজাম/আটাশ'
 'চাল (মোটা)/স্বর্ণা/চায়না ইরি' 'চাল (মাঝারী)পাইজাম/লতা']
['মশুর ডাল (বড় দানা)' 'মশুর ডাল (মাঝারী দানা)' 'মশুর ডাল (ছোট দানা)'
 'মুগ ডাল (মানভেদে)' 'এ্যাংকর ডাল' 'ছোলা (মানভেদে)' 'আলু (মানভেদে)'
 'মশূর ডাল (মাঝারী দানা)' 'আলু (নতুন, মানভেদে)'
 'আলু (নতুন/পুরাতন)(মানভেদে)']
['চিনি' 'খেজুর(সাধারণ মানের)' 'লবণ(প্যাঃ)আয়োডিনযুক্ত' 'ডিম (ফার্ম)'
 'লেখার কাগজ(সাদা)' 'এম,এস রড (৬০ গ্রেড)' 'এম,এস রড( ৪০ গ্রেড)'
 'লবণ(প্যাঃ)আয়োডিনযুক্ত(মানভেদে)']
['সয়াবিন তেল (লুজ)' 'সয়াবিন তেল (বোতল)' 'পাম অয়েল (লুজ)'
 'সুপার পাম অয়েল (লুজ)' 'রাইস ব্রান তেল (বোতল)' 'পাম অয়েল (সুপার)'
 'পাম অলিন (বোতল)']
['পিঁয়াজ (দেশী)' 'পিঁয়াজ (আমদানি)' 'রসুন (দেশী)' 'রসুন (আমদানি)'
 'শুকনা মরিচ (দেশী)' 'শুকনা মরিচ (আমদানি)' 'হলুদ (দেশী)' 'হলুদ (আমদানি)'
 'আদা (দেশী)' 'আদা (আমদানি)' 'জিরা' 'দারুচিনি' 'লবঙ্গ' 'এলাচ(ছোট)' 'ধনে'
 'তেজপাতা' 'রসুন(দেশী) নতুন/পুর

**The categorization seems alright**

In [ ]:
logger.info("=== PROCESSING SUMMARY ===")
    logger.info(f"Total files processed: {files_processed}")
    logger.info(f"Files with data extracted: {files_with_data}")
    logger.info(f"Files with NO data extracted: {files_without_data}")
    logger.info(f"Files with errors: {files_error}")
